# Lab 8: Thực hành chuyên sâu về Phân tích cú pháp phụ thuộc

## Phần 2: Phân tích câu và Trực quan hóa

In [ ]:
import spacy
from spacy import displacy
# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

In [ ]:
# Tùy chọn để hiển thị trong trình duyệt
options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
displacy.serve(doc, style="dep")

/usr/local/lib/python3.12/dist-packages/spacy/displacy/__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
for token in doc:
    if token.dep_ == "ROOT":
        print(f"Từ gốc (ROOT) trong câu '{doc}' là : '{token.text}'")

Từ gốc (ROOT) trong câu 'The quick brown fox jumps over the lazy dog.' là : 'jumps'


In [ ]:
target = doc[4]
print("Từ:", target.text)
print("Root:", target.head.text)
print("Quan hệ với root:", target.dep_)

print("\nCác phụ thuộc của token:")
for child in target.children:
    print(f"- {child.text:10}  | dep = {child.dep_}")

Từ: jumps
Root: jumps
Quan hệ với root: ROOT

Các phụ thuộc của token:
- fox         | dep = nsubj
- over        | dep = prep
- .           | dep = punct


In [ ]:
target = doc[3]
print(f"\nNhững từ mà '{target.text}' làm head:")
for child in target.children:
    print(f"- {child.text:10}  | dep = {child.dep_}")


Những từ mà 'fox' làm head:
- The         | dep = det
- quick       | dep = amod
- brown       | dep = amod


## Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [ ]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    # Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


- `token.text`: Văn bản của token
- `token.dep_`: Nhãn quan hệ phụ thuộc của token này với head của nó
- `token.head.text`: Văn bản của token `head`
- `token.children`: Một iterator chứa các token con (dependent) của token hiện tại.

## Phần 4: Duyệt cây phụ thuộc đẻ trích xuất thông tin

### 4.1. Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [ ]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        obj = ""
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
            if subject and obj:
                print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (cat, chased, mouse)
Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)
Found Triplet: (dog, watched, them)


### 4.2. Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [ ]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)
for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        adjectives = []
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


## Phần 5: Bài tập tự luyện

### Bài 1: Tìm động từ chính của câu

In [ ]:
def find_main_verb(doc):
    """
    Trả về động từ chính (ROOT) trong câu.
    Nếu không tìm thấy ROOT phù hợp, trả về None.
    """
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token

    # Trường hợp hiếm: ROOT không phải VERB (ví dụ câu mệnh lệnh, câu bị động)
    for token in doc:
        if token.dep_ == "ROOT":
            return token

    return None

doc = nlp("The quick brown fox jumps over the lazy dog.")
print(f"Động từ chính của câu '{doc.text}' là:",find_main_verb(doc))

Động từ chính của câu 'The quick brown fox jumps over the lazy dog.' là: jumps


### Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

In [ ]:
def extract_noun_chunks(doc):
    """
    Tự trích xuất các cụm danh từ mà không dùng doc.noun_chunks.
    Ý tưởng:
    - Chọn các token là NOUN hoặc PROPN
    - Lấy các từ bổ nghĩa thuộc nhóm (det, amod, compound, nummod, poss)
    - Lấy thêm các từ mở rộng bên trái/phải
    """

    noun_chunks = []

    # Các quan hệ bổ nghĩa thường gặp của danh từ
    modifiers = {"det", "amod", "compound", "nummod", "poss"}

    for token in doc:
        if token.pos_ in ("NOUN", "PROPN"):
            # lấy ranh giới trái và phải của cụm
            left = token.i
            right = token.i

            # mở rộng sang trái (từ bổ nghĩa trước danh từ)
            for child in token.children:
                if child.dep_ in modifiers and child.i < token.i:
                    left = min(left, child.i)

            # mở rộng sang phải (ít gặp nhưng có thể có: ví dụ noun + possessive)
            for child in token.children:
                if child.dep_ in modifiers and child.i > token.i:
                    right = max(right, child.i)

            span = doc[left:right+1]
            noun_chunks.append(span.text)

    return noun_chunks

doc = nlp("The quick brown fox jumps over the lazy dog near a small river.")
print(f"Các cụm danh từ trong câu '{doc.text}' là:\n {extract_noun_chunks(doc)}")

Các cụm danh từ trong câu 'The quick brown fox jumps over the lazy dog near a small river.' là:
 ['The quick brown fox', 'the lazy dog', 'a small river']


### Bài 3: Tìm đường đi ngắn nhất trong cây

In [ ]:
def get_path_to_root(token):
    """
    Trả về đường đi từ token hiện tại lên đến ROOT.
    Đường đi là một list token, bắt đầu từ token và kết thúc ở ROOT.
    """
    path = [token]
    current = token

    # Duyệt lên cây cho đến khi gặp ROOT
    while current.dep_ != "ROOT":
        current = current.head
        path.append(current)

    return path
token = nlp("The quick brown fox jumps over the lazy dog")[2]
print(f"Danh sách các token trên đường đi từ '{token.text}' đến ROOT là:\n {get_path_to_root(token)}")
# print("Đường đi token đến ROOT:", " -> ".join([t.text for t in get_path_to_root(token)]))

Danh sách các token trên đường đi từ 'brown' đến ROOT là:
 [brown, fox, jumps]
